## Calculating and Plotting a Composite With Significance

What is a composite?  mean of a field conditioned on the value of another field.

_Example:_

We have heard that ENSO changes atmospheric circulation and impacts precipitation and temperature globally.  We can explore this using composites. 

_Specific statement of the problem:_ 

What are the precipitation anomalies during El Nino vs. Neutral vs. La Nina _and how are they different_?

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np

import cartopy.crs as ccrs
import cartopy.mpl.ticker as cticker
from cartopy.util import add_cyclic_point

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

### ENSO

We have an index called Nino3.4 that quantifies ENSO.  When it is large and positive, we say there is an El Nino.  When it is large and negative, we say there is a La Nina.  In between, we say it is neutral.

Nino3.4 is calculated at the SST anomalies in a particular region in the Tropical Pacific.  We calculated this previously using the NOAA OISST data and then Subsetting, Groupby, and Aggregating. I wrote this data to a file:

`/scratch/kpegion/nino34_1982-2019.oisstv2_anoms.nc`

In [ ]:
file_nino34='/home/pdirmeye/classes/clim680_2022/nino34_1982-2019.oisstv2_anoms.nc'
ds_nino34=xr.open_dataset(file_nino34)
ds_nino34

This data goes from 1982-2019

We can plot it as a timeseries

### Defining El Nino, La Nina, and Neutral

Nino3.4 >= 1 -> El Nino

Nino3.4 <= -1 -> La Nina

Nino3.4 > -1 and Nino3.4 < 1 -> Neutral

In [ ]:
elnino=ds_nino34['sst'].where(ds_nino34['sst']>=1)
lanina=ds_nino34['sst'].where(ds_nino34['sst']<=-1)
neutral=ds_nino34['sst'].where(np.logical_and(ds_nino34['sst']>-1,
                                              ds_nino34['sst']<1))

print('El Nino: ',elnino.count(dim='time').values)
print('Neutral: ',neutral.count(dim='time').values)
print('La Nina: ',lanina.count(dim='time').values)

counts=[elnino.count(dim='time').values,
        lanina.count(dim='time').values,
        neutral.count(dim='time').values]
print(counts)

plt.plot(ds_nino34['time'],ds_nino34['sst'],'k')
plt.fill_between(ds_nino34['time'].values,ds_nino34['sst'],color='lightgreen')
plt.fill_between(ds_nino34['time'].values,elnino,y2=1.0,color='red')
plt.fill_between(ds_nino34['time'].values,lanina,y2=-1.0,color='blue')
plt.axhline(0,color='black',linewidth=0.5)
plt.axhline(1,color='black',linewidth=0.5,linestyle='dotted')
plt.axhline(-1,color='black',linewidth=0.5,linestyle='dotted') ;

### Precipitation Data

We will use the Global Precipitation Climatology Project (GPCP) Monthly Precipitation Data located on the COLA computers:

https://kpegion.github.io/COLA-DATASETS-CATALOG/gpcp_precip.mon.mean.nc

In [ ]:
file='/home/pdirmeye/classes/clim680_2022/GPCP_precip.mon.mean.nc'
ds_precip = xr.open_dataset(file)
ds_precip

This data goes from 1979-2020.  Let's select the same times as the nino34 data.

In [ ]:
da_precip = ds_precip.precip.sel(time=slice(ds_nino34['time'][0],ds_nino34['time'][-1]))

da_climo = da_precip.groupby('time.month').mean()
da_anoms = da_precip.groupby('time.month')-da_climo
da_anoms

### Now we can select the dates that match El Nino, La Nina, and Neutral

In [ ]:
elnino_precip = da_anoms.sel(time=elnino.dropna(dim='time')['time']).mean(dim='time')
lanina_precip = da_anoms.sel(time=lanina.dropna(dim='time')['time']).mean(dim='time')
neutral_precip = da_anoms.sel(time=neutral.dropna(dim='time')['time']).mean(dim='time')

comp_precip = [elnino_precip,lanina_precip,neutral_precip]
comp_precip

In [ ]:
labels=['El Nino','La Nina', 'Neutral']
clevs = np.arange(-2.0,2.1,0.25)

# Define the figure and each axis for the 3 rows and 3 columns
fig, axs = plt.subplots(nrows=3,ncols=1,
                        subplot_kw={'projection': ccrs.PlateCarree()},
                        figsize=(8.5,11))

# axs is a 2 dimensional array of `GeoAxes`.  
# We will flatten it into a 1-D array (just 3 rows)
axs = axs.flatten()

#Loop over all of the seasons and plot
for i,enso in enumerate(comp_precip):

        # Select the season
        data = comp_precip[i]

        # Add the cyclic point
        data,lons = add_cyclic_point(data,coord=comp_precip[i]['lon'])

        # Contour plot
        cs=axs[i].contourf(lons,comp_precip[i]['lat'],data,clevs,
                          transform = ccrs.PlateCarree(),
                          cmap='BrBG',extend='both')

       # Longitude labels
        axs[i].set_xticks(np.arange(-180,181,60), crs=ccrs.PlateCarree())
        lon_formatter = cticker.LongitudeFormatter()
        axs[i].xaxis.set_major_formatter(lon_formatter)

        # Latitude labels
        axs[i].set_yticks(np.arange(-90,91,30), crs=ccrs.PlateCarree())
        lat_formatter = cticker.LatitudeFormatter()
        axs[i].yaxis.set_major_formatter(lat_formatter)

        
        # Title each subplot with the name of the season
        axs[i].set_title(labels[i]+' ('+str(counts[i])+')')

        # Draw the coastines for each subplot
        axs[i].coastlines()
        
# Adjust the location of the subplots 
# on the page to make room for the colorbar
fig.subplots_adjust(bottom=0.25, top=0.9, left=0.05, right=0.95,
                    wspace=0.1, hspace=0.5)

# Add a colorbar axis at the bottom of the graph
cbar_ax = fig.add_axes([0.25, 0.18, 0.5, 0.012])

# Draw the colorbar
cbar = fig.colorbar(cs,cax=cbar_ax,orientation='horizontal',label='mm/day')

# Add a big title at the top
plt.suptitle('Composite Precipitation Anomalies during ENSO') ;

--------------
--------------
## Significance Testing

We want to know where the El Niño or La Niña composites are truly different from the Neutral composite.  

Last week we discussed the issue of trying to compare our neutral composite with our El Niño or La Niña composite, but we have issues of sample size.  

* Neutral has 330 values
* El Niño has 61 values
* La Niña has 65 values

Many more values went into calculating the mean for neutral. 

One solution to address this is to calculate statistical significance.

### First, let's calculate differences

In [ ]:
diff_precip=[elnino_precip-neutral_precip, 
             lanina_precip-neutral_precip]

### Now plot our difference maps

Make the difference plots first and think about what they tell you scientifically before calculating and adding significance to them. 

Significance is a way of quantifying whether your results occurred by random chance.  

If you don't have a scientific result, then significance will not create one.   

In [ ]:
labels=['El Nino-Neutral','La Nina-Neutral']
clevs=np.arange(-3,3.5,0.5)

# Define the figure and each axis for the 3 rows and 3 columns
fig, axs = plt.subplots(nrows=2,ncols=1,
                        subplot_kw={'projection': ccrs.PlateCarree()},
                        figsize=(8.5,11))

# axs is a 2 dimensional array of `GeoAxes`.  
# We will flatten it into a 1-D array
axs = axs.flatten()

#Loop over all of the seasons and plot
for i,enso in enumerate(diff_precip):

        # Select the season
        data = diff_precip[i]

        # Add the cyclic point
        data,lons = add_cyclic_point(data,coord=diff_precip[i]['lon'])

        # Contour plot
        cs = axs[i].contourf(lons,diff_precip[i]['lat'],data,clevs,
                          transform=ccrs.PlateCarree(),
                          cmap='BrBG',extend='both')

       # Longitude labels
        axs[i].set_xticks(np.arange(-180,181,60), crs=ccrs.PlateCarree())
        lon_formatter = cticker.LongitudeFormatter()
        axs[i].xaxis.set_major_formatter(lon_formatter)

        # Latitude labels
        axs[i].set_yticks(np.arange(-90,91,30), crs=ccrs.PlateCarree())
        lat_formatter = cticker.LatitudeFormatter()
        axs[i].yaxis.set_major_formatter(lat_formatter)

        
        # Title each subplot with the name of the season
        axs[i].set_title(labels[i])

        # Draw the coastines for each subplot
        axs[i].coastlines()
        
# Adjust the location of the subplots 
# on the page to make room for the colorbar
fig.subplots_adjust(bottom=0.25, top=0.9, left=0.05, right=0.95,
                    wspace=0.1, hspace=0.5)

# Add a colorbar axis at the bottom of the graph
cbar_ax = fig.add_axes([0.25, 0.18, 0.5, 0.012])

# Draw the colorbar
cbar = fig.colorbar(cs, cax=cbar_ax,orientation='horizontal',label='mm/day')

# Add a big title at the top
plt.suptitle('Composite Precipitation Differences during ENSO')

### Calculate Significance Using the built-in Stats Functions

Python has a library of functions called `scipy`, which stands for "Science Python". 
It includes many packages and functions of use in science, which are more _heavy duty_ than what most
non-scientist users need. 
One of those packages is `scipy.stats`, which includes many common (and uncommon) statistical functions.

A major element of Climate Science is testing whether a result is _statistically significant_, i.e., 
significantly different that what might occur due to expected natural variations; different from _chance_.
You will have a difficult time convincing anyone that your research conclusions are robust and meaningful
unless you demonstrate they are _statistically significant_.

To do this correctly, you need to think carefully about your analysis and define precisely your 
problem and the meaning of its possible outcomes. 
This is typically done by defining a **null hypothesis** that defines a lack of significance, and then
testing whether it can be disproved.

__State our problem clearly__

Where is the mean difference different enough to be considered significant (meaning its unlikely it happened by random chance)?

$\mu_x$ = El Nino Composite ($n_x$=61)

$\mu_y$ = Neutral Composite ($n_y$=330)

__Define our Null Hypothesis__

Null Hypothesis (H0): Differences between the two means is zero

$\mu_x-\mu_y=0$

__Pick a level of significance__

Pick a p-value (or level of significance).  Typically, 1% or 5% is chosen.

You must pick this level first. 

* It is cheating to pick it after you see the results. 
* It is also cheating to make changes to your analysis to get a desirable p-value.

Doing things to your data or analysis to get a higher p-value is 
called [P-hacking](https://www.nature.com/news/scientific-method-statistical-errors-1.14700)

In [ ]:
p = 0.05 # 95% confidence level, or 5% likelihood that the result arose by chance

### Use the stats `ttest_ind` function

Let's follow the documentation to understand what this function does...

[ttest_ind](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html)

This is a two-sided test, based on the __Student's t-Test__, for the null hypothesis that 2 independent samples have identical average (expected) values. This test assumes that the populations have identical variances by default.

#### Input: 

**Two arrays (a,b) to compare the means and perform a ttest**
* a = values that go into elnino_precip composite 
* b = values that go into neutral_precip

We can set several other optional parameters for this function:

**axis:  Axis along which to compute test. If None, compute over the whole arrays, a, and b.**

* We want this calculated over our whole grid/array, so no need to specify

**equal_var: If True (default), perform a standard independent 2 sample test that assumes equal population variances [1]. If False, perform Welch’s t-test, which does not assume equal population variance [2].**

* We probably don't have equal variance, so set this to `False`

**nan_policy: {‘propagate’, ‘raise’, ‘omit’}, optional
Defines how to handle when input contains nan. The following options are available (default is ‘propagate’):**

**`propagate`: returns nan**

**`raise`: throws an error**

**`omit`: performs the calculations ignoring nan values**

* We can leave this as default

#### Output 

**`t-statistic`: The calculated t-statistic.**

**`pvalue`: The two-tailed p-value. This is the p-value telling us the % chance we have in being incorrect if we reject the null hypothesis. This is the information we want. We will identify where this `pvalue` is less than our `p=0.05`.**


In [ ]:
from scipy.stats import ttest_ind

elnino_precip_vals = da_anoms.sel(time=elnino.dropna(dim='time')['time'])
lanina_precip_vals = da_anoms.sel(time=lanina.dropna(dim='time')['time'])
neutral_precip_vals = da_anoms.sel(time=neutral.dropna(dim='time')['time'])

In [ ]:
tstat,pval = ttest_ind(elnino_precip_vals, 
                neutral_precip_vals, 
                equal_var = False)

In [ ]:
plt.contourf(pval,[0,0.05,1],cmap='RdPu_r') 
plt.colorbar(orientation='horizontal') ;

### Make a mask of where the value is significant

In [ ]:
mask_diff = diff_precip[0].where(pval <= p)
mask_diff

## Plot our difference between composites and also mark where its significant

In [ ]:
labels=['El Nino vs. Neutral']
clevs=np.arange(-3,3.5,0.5)
hatch_levs=[0]

# Define the figure and each axis for the 2 rows and 1 columns
fig, axs = plt.subplots(nrows=1,ncols=1,
                        subplot_kw={'projection': ccrs.PlateCarree(central_longitude=200.0)})


data=diff_precip[0]

# Add the cyclic point
data,lons=add_cyclic_point(data,coord=diff_precip[0]['lon'])
mask_data,lons=add_cyclic_point(mask_diff,coord=diff_precip[0]['lon'])

# Contour plot
cs=axs.contourf(lons,diff_precip[0]['lat'],data,clevs,
                transform = ccrs.PlateCarree(),
                cmap='BrBG',extend='both')

# Overlay of stippling
ss=axs.contourf(lons,diff_precip[0]['lat'],mask_data,[0,1],
                transform = ccrs.PlateCarree(),colors=['None'],
                hatches=['.',''],extend='both',alpha=0.0)

# Draw the coastines for each subplot
axs.coastlines()
        
# Adjust the location of the subplots 
# on the page to make room for the colorbar
fig.subplots_adjust(bottom=0.25, top=0.9, left=0.05, right=0.95,
                    wspace=0.1, hspace=0.5)

# Add a colorbar axis at the bottom of the graph
cbar_ax = fig.add_axes([0.15, 0.18, 0.7, 0.025])

# Draw the colorbar
cbar = fig.colorbar(cs,cax=cbar_ax,orientation='horizontal',label='mm/day')

# Add a big title at the top
plt.suptitle('Composite Precipitation Differences El Nino-Neutral')

plt.savefig('stats_ttest.png')

### Another Way to Present Significance Data

Using semi-transparent shading to mask insignficant areas

In [ ]:
mask_diff_o = xr.ones_like(diff_precip[0]).where(pval > p) # This retains data where the pval is insignificant


labels=['El Nino vs. Neutral']
clevs=np.arange(-3,3.5,0.5)
hatch_levs=[0]

# Define the figure and each axis for the 2 rows and 1 columns
fig, axs = plt.subplots(nrows=1,ncols=1,
                        subplot_kw={'projection': ccrs.PlateCarree(central_longitude=200.0)})

data=diff_precip[0]

# Add the cyclic point
data,lons=add_cyclic_point(data,coord=diff_precip[0]['lon'])
mask_data_o,lons=add_cyclic_point(mask_diff_o,coord=diff_precip[0]['lon'])

# Contour plot
cs=axs.contourf(lons,diff_precip[0]['lat'],data,clevs,
                transform = ccrs.PlateCarree(),
                cmap='BrBG',extend='both')

# Mask out insignficant areas with semi-transparent shading
ss=axs.contourf(lons,diff_precip[0]['lat'],mask_data_o,
                transform = ccrs.PlateCarree(),colors=['k'],
                alpha=0.3)

# Draw the coastines for each subplot
axs.coastlines()
        
# Adjust the location of the subplots 
# on the page to make room for the colorbar
fig.subplots_adjust(bottom=0.25, top=0.9, left=0.05, right=0.95,
                    wspace=0.1, hspace=0.5)

# Add a colorbar axis at the bottom of the graph
cbar_ax = fig.add_axes([0.15, 0.18, 0.7, 0.025])

# Draw the colorbar
cbar = fig.colorbar(cs,cax=cbar_ax,orientation='horizontal',label='mm/day')

# Add a big title at the top
plt.suptitle('Composite Precipitation Differences El Nino-Neutral')

plt.savefig('stats_ttest.png',dpi=200)

-------------
### Non-parametric significance testing
The use of statistics like standard deviation to estimate significance is common practice, but it is based on assumptions.
Specifically, that the data being examined has the following properties:
1. The data have a [normal distribution](https://en.wikipedia.org/wiki/Normal_distribution)
2. The important relationships between data sets are [linear](https://en.wikipedia.org/wiki/Linearity)

We will talk more about linearity later, when we discuss regressions and correlation. But the Student's t-Test we used above makes the assumption that anomalies in the data are normally distributed.  This is often a good assumption for many climate variables (e.g., temperature). However, it is not so good for other, e.g., precipitation, especially when the data are sampled on a short time interval (e.g., daily). What about the monthly data examined here?  Well, let's take a look:

In [ ]:
from scipy.stats import norm

#lat,lon = 39,-79
lat,lon = -30,140
grid_anoms = da_anoms.sel(lat=lat,lon=lon,method='nearest')

spacing = 0.2
xvals = np.arange(-4,4.1,spacing)
p_distrib = plt.hist(grid_anoms,bins=xvals,label="Observed")

plt.plot(xvals,spacing*grid_anoms.count().data*norm.pdf(xvals,grid_anoms.mean(),grid_anoms.std()),label="Normal Dist.")
plt.legend()

plt.ylabel("Count [months]")
plt.xlabel("Anomalies [mm/day]")

plt.axvline(0,color='w',linewidth=0.5)

slon = f"{lon}˚E"
if lon < 0: slon=f"{-lon}˚W"
slat = f"{lat}˚N"
if lat < 0: slat=f"{-lat}˚S"

plt.title(f"Distribution of Monthly Precipitation Anomalies at {slat} {slon}") ;

Here you can see that the precipitation distribution is [skewed](https://en.wikipedia.org/wiki/Skewness) positive (i.e., the long tail is on the positive side of the distribution and the median is less than the mean). This is common for precipitation data.

What can we do?

### Nonparametric statistics

Rather than assume a distribution for the data, we can query the data directly to find where an anomaly ranks among all the samples in the dataset. This is a form of [nonparameteric statistics](https://en.wikipedia.org/wiki/Nonparametric_statistics) where no assumptions are made. This also makes conclusions more robust. With today's computing power, it is often fairly easy to test the significance of an anomaly by simply comparing its magnitude to available past data.

For this location, let's find the likelihood of a monthly precipitation anomaly greater than 2 mm/day. 

In [ ]:
test_val = 2.0

# Based on the assumption of a normal distribution:
norm_prob = norm.sf(test_val,grid_anoms.mean(),grid_anoms.std())

# Based on the actual data distribution:
epsilon = 1e-6
test_bin = (np.argwhere(abs(p_distrib[1] - test_val) < epsilon)[0][0]) # Find histogram bin corresponding to the the test value
data_prob = 1.0 - p_distrib[0][:test_bin].sum() / grid_anoms.count().data

print(f"Probability of >2 mm/day monthly anomaly at {slat} {slon}: \
       \nNormal data distribution = {100*norm_prob:0.2f}% \
       \nActual data distribution = {100*data_prob:0.2f}%")

We can flip the question around and ask at what anomaly value would we expect only a 1% chance of 
precipitation rates exceeding that value for the month (i.e., 99% chance precipitation rates are below the value). 
This uses the `quantile` method in `numpy`: 

In [ ]:
chance = 0.99

# Based on the assumption of a normal distribution:
norm_value = norm.ppf(chance,grid_anoms.mean(),grid_anoms.std())

# Based on the actual data distribution:
data_value = np.quantile(grid_anoms,chance)

print(f"Rainfall rate corresponding to the {100*chance:0.0f}th percentile at {slat} {slon}: \
       \nNormal data distribution = {norm_value:0.2f} mm/day \
       \nActual data distribution = {data_value:0.2f} mm/day")